In [2]:
# !pip install -q matplotlib

In [3]:
# TODO: Change this as required
CHUNK_NUM = "03"
prefix = "shard_"
input_f = prefix + CHUNK_NUM
output_dir = 'outputs/'

output_f = output_dir  + "output_" + prefix + CHUNK_NUM + '.csv'

In [4]:
import pandas as pd
import re

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import pickle
from typing import List

import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
tqdm.pandas()

In [5]:
df = pd.read_json(input_f, lines=True)

In [6]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters like &gt;
    text = re.sub(r'&\w+;', '', text)
    return text.lstrip('1234567890')

In [7]:
import time
import pickle

cnt = 0
total = 0

start_time = time.time()

partial_rows = []

for i in range(df.shape[0]):
  for j in range(len(df['posts'][i])):
    total += 1
    if 'perspectives' not in df['posts'][i][j] or 'com' not in df['posts'][i][j] or 'no' not in df['posts'][i][j] or 'now' not in df['posts'][i][j]:
      continue

    ps = df['posts'][i][j]['perspectives']
    c = clean_text(df['posts'][i][j]['com'])
    no = df['posts'][i][j]['no']
    ts = df['posts'][i][j]['now']

    if len(c) < 15:
      continue

    if max(ps.values()) >= 0.5:
      continue

    if 'http' in c:
      continue

    transformed_row = {k: v for k, v in ps.items()}
    transformed_row['com'] = c
    transformed_row['no'] = no
    transformed_row['ts'] = ts

    partial_rows.append(transformed_row)
    cnt += 1
    # print(ps)
    # print(c)
    # print(no)
    # print(ts)

transformed_df = pd.DataFrame(partial_rows)
transformed_df.to_csv(input_f + '_transformed.csv', index_label = 'id')

#pickle.dump(partial_rows, open(input_f + '_transformed.pkl', 'wb'))
end_time = time.time()

print(cnt)
print(total)

secs = end_time - start_time
per_entry = secs / total
print(f"For 100k entries it should take: {100000 * per_entry}")
print(f"Filtered out {100.0 * (total - cnt) / total} % of entries")


71504
255412
For 100k entries it should take: 5.425520328422975
Filtered out 72.00444771584733 % of entries


In [8]:
final_df = pd.read_csv(input_f + '_transformed.csv')
final_df

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts
0,0,0.230453,0.066812,0.127698,0.110461,0.198860,0.025815,0.070034,LIVE interview with Cornel West @ 3:30 ET on J...,81462958,07/17/16(Sun)13:06:20
1,1,0.198800,0.181339,0.099273,0.117121,0.085800,0.230993,0.339036,"Bump, Im gonna estch the video",81459486,07/17/16(Sun)12:39:14
2,2,0.070573,0.031869,0.150659,0.037606,0.072772,0.033200,0.085553,Makes a lot of sense,81460106,07/17/16(Sun)12:43:55
3,3,0.074633,0.033033,0.112461,0.035503,0.074861,0.009613,0.018216,is there examples of this actually happening i...,81460260,07/17/16(Sun)12:45:04
4,4,0.388838,0.199218,0.433990,0.164136,0.434326,0.066841,0.073042,"I dont know, man. He makes some compelling arg...",81461604,07/17/16(Sun)12:55:52
...,...,...,...,...,...,...,...,...,...,...,...
71499,71499,0.122384,0.080110,0.147145,0.086850,0.121551,0.117746,0.281687,You made it bro,81749831,07/19/16(Tue)09:55:09
71500,71500,0.447778,0.407818,0.225239,0.441723,0.489485,0.179133,0.445306,Danke Mama merkel,81749870,07/19/16(Tue)09:55:34
71501,71501,0.084574,0.067248,0.123328,0.086614,0.080826,0.056194,0.439385,Forever alone rio 2016,81750546,07/19/16(Tue)10:02:28
71502,71502,0.214657,0.204825,0.102508,0.229995,0.249678,0.492889,0.133692,Bror....broder is plural,81752619,07/19/16(Tue)10:23:26


In [9]:
# Load the model and tokenizer once
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.padding_side = "left"
# tokenizer.pad_token = '<pad>'
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

def get_dialogpt_response(comment):
    # Tokenize the input comment
    input_ids = tokenizer.encode(comment + tokenizer.eos_token, return_tensors="pt").to('cuda')

    # Generate a response
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

In [10]:
import pandas as pd

def process_and_save(final_df, func, b_chunk_size=200):
    """Applies a function to the DataFrame in chunks and saves periodically."""

    for i in range(0, len(final_df), b_chunk_size):
        chunk = final_df.iloc[i:i + b_chunk_size]
        chunk['response'] = chunk['com'].progress_apply(func)

        # Append to CSV file (mode='a' for append)
        chunk.to_csv(output_f, mode='a', header=i == 0, index=False)

process_and_save(final_df, get_dialogpt_response)

  0%|          | 0/200 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/scratch/1028977/ipykernel_2155203/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2155203/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

...

/scratch/1028977/ipykernel_2155203/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2155203/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
sanity = pd.read_csv(output_f)
sanity

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts,response
0,0,0.230453,0.066812,0.127698,0.110461,0.198860,0.025815,0.070034,LIVE interview with Cornel West @ 3:30 ET on J...,81462958,07/17/16(Sun)13:06:20,NaN
1,1,0.198800,0.181339,0.099273,0.117121,0.085800,0.230993,0.339036,"Bump, Im gonna estch the video",81459486,07/17/16(Sun)12:39:14,I'm gonna estch the video
2,2,0.070573,0.031869,0.150659,0.037606,0.072772,0.033200,0.085553,Makes a lot of sense,81460106,07/17/16(Sun)12:43:55,I'm glad you think so!
3,3,0.074633,0.033033,0.112461,0.035503,0.074861,0.009613,0.018216,is there examples of this actually happening i...,81460260,07/17/16(Sun)12:45:04,"I'm not sure, but I think it's a common occurr..."
4,4,0.388838,0.199218,0.433990,0.164136,0.434326,0.066841,0.073042,"I dont know, man. He makes some compelling arg...",81461604,07/17/16(Sun)12:55:52,"I think it's a good theory, but I think it's a..."
...,...,...,...,...,...,...,...,...,...,...,...,...
71499,71499,0.122384,0.080110,0.147145,0.086850,0.121551,0.117746,0.281687,You made it bro,81749831,07/19/16(Tue)09:55:09,Thanks bro
71500,71500,0.447778,0.407818,0.225239,0.441723,0.489485,0.179133,0.445306,Danke Mama merkel,81749870,07/19/16(Tue)09:55:34,Danke Mama Merkel
71501,71501,0.084574,0.067248,0.123328,0.086614,0.080826,0.056194,0.439385,Forever alone rio 2016,81750546,07/19/16(Tue)10:02:28,I'm not alone!
71502,71502,0.214657,0.204825,0.102508,0.229995,0.249678,0.492889,0.133692,Bror....broder is plural,81752619,07/19/16(Tue)10:23:26,I'm aware.
